In [15]:
import os
import glob
import pandas as pd
import numpy as np
import logging
import umap.umap_ as umap
import IPython.display as display
import ipywidgets as widgets
from pathlib import Path
from tqdm.notebook import tqdm
from collections import defaultdict
from cycler import cycler
from PIL import Image

from typing import List, Tuple, Union

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision.io import ImageReadMode, read_image
from torchvision import transforms
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as LP
from pytorch_metric_learning import losses, miners, samplers, testers, trainers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.utils.inference import InferenceModel, MatchFinder
from pytorch_metric_learning.distances import CosineSimilarity

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.getLogger().setLevel(logging.INFO)
logging.info("VERSION %s" % pytorch_metric_learning.__version__)

/Users/julia/venvs/python3.9/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/julia/venvs/python3.9/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/julia/venvs/python3.9/lib/python3.9/site-packages/umap/distances.py:1086: Num

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Описание датасета

- Класс изображения - id внутри имени файла, он же по сути instance объекта, например, id товара вендора
- Классов > половины длины датасета, то есть классическая классификация не пойдет, тк слой классификации будет слишком большой, а также если добавится новый класс в данные, то нужно переучивать модель
- Данных на класс ~1.5 изображения

## Подход
- Выберем embedded подход для построения вероятности "идентичности" двух изображений, то есть будем учить модель предсказывать близкие эмбеддинги для похожих изображений и наоборот
- Так как кластера у нас по сути это инстанс объекта, то размер кластера будет очень маленький, поэтому для нас важно разнести кластера на приличное расстояние друг от друга, для это будем использовать лосс с допуском (margin)
- В результате получая на вход две картинки, мы производит препроцессинг, прогоняем через модель и получаем два эмбеддинга
- Для получения финальной вероятности возьмем косинусное расстояние между эмбеддингами и переведем его из [-1, 1] в [0, 1], что будет равносильно распределению вероятностей

In [11]:
# Посмотрим на изображения
image_paths = sorted(list(Path('test-task/clusters/').iterdir()))
data = pd.read_csv('test-task/clusters.csv', index_col='Unnamed: 0')

i = 0
while i < 6:
    # skip objects which contain less than 2 images
    if str(image_paths[i]).split('_')[0] != str(image_paths[i + 1]).split('_')[0]:
        i += 1
        continue
        
    image1 = open(image_paths[i],'rb').read()
    image2 = open(image_paths[i+1], 'rb').read()
    
    wi1 = widgets.Image(value=image1, format='jpg', width=300, height=400)
    wi2 = widgets.Image(value=image2, format='jpg', width=300, height=400)
    a = [wi1, wi2]
    wid = widgets.HBox(a)
    print(image_paths[i], image_paths[i + 1])
    display.display(wid)

    i += 2

## Параметры модели и обучения

In [19]:
num_epochs = 10
dataloader_num_workers = 2

model_name ='EfficientNet_B3'
image_size = (320, 300) # специфичный размер для EfficientNet_B3
embedding_size = 256 
batch_size = 12

# Создаем Дата класс

In [17]:
def get_train_transforms() -> transforms.Compose:
    return transforms.Compose(
            [
                transforms.Resize(size=image_size, interpolation=transforms.InterpolationMode.BILINEAR),
                transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                
            ]
    )

def get_valid_transforms() -> transforms.Compose:
    return transforms.Compose(
            [
                transforms.Resize(size=image_size, interpolation=transforms.InterpolationMode.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),                
            ]
    )

In [7]:
class ImageDataset(Dataset):
    '''
    Input:
        data: DataFrame with image_path and label columns
        transforms: image trainsformations, resize included
    '''
    
    def __init__(
        self,
        image_dir: Path,
        csv_layout: pd.DataFrame,
        transforms: transforms.Compose = None,
        return_labels=True,
        verify_images=False
    ):
        self.image_dir = image_dir   
        self.csv_layout = csv_layout
        
        self.image_to_label = None
        self.label_to_images = None
        self.csv_to_labels()
        
        self.images = list(self.csv_layout['file_name'])
        self.images_paths = [Path(self.image_dir, image) for image in self.images]
        self.labels = list(self.csv_layout['label'])
        
        if verify_images:
            self.verify_images()
        
        self.augmentations = transforms
        self.return_labels = return_labels
        
        assert self.image_to_label, "There is no labels in data" 
        
    def csv_to_labels(self) -> None:
        '''
        Сonvert Pandas.DataFrame with claster - image columns
        to dict image->label (i.e class of image)
        '''
        assert 'cluster_id' in self.csv_layout, "cluster_id not in csv_layout" 
        assert 'file_name' in self.csv_layout, "file_name not in csv_layout" 
        assert 'label' in self.csv_layout, "label not in csv_layout" 
        
        self.image_to_label = defaultdict()
        self.label_to_images = defaultdict(list)
        
        clusters = self.csv_layout['label']
        images = self.csv_layout['file_name']
        
        for class_id, filename in zip(clusters, images):
            self.image_to_label[filename] = class_id
            self.label_to_images[class_id].append(filename)
            
            
    def verify_images(self) -> None:
        valid_images = 0
        
        logging.info("Start image validation")
        for image_path in tqdm(self.images_paths):
            try:
                image = Image.open(image_path).convert('RGB')
                valid_images += 1
                
            except Exception as e:
                print(f'corrupted image: {image_path}', e)
                
        logging.info(f"Valid data: {valid_images}, {100 * valid_images/len(self.images_paths)}%")
        logging.info(f"Corrupted data: {len(self.images_paths) - valid_images}, {100 * (1 - valid_images/len(self.images_paths))}%")
                
        
    def __len__(self) -> int:
        return len(self.images_paths)

    def __getitem__(self, idx) -> Union[torch.tensor, List[torch.tensor]]:
        
        image_path = self.images_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        if self.augmentations:
            image = self.augmentations(image)
        
        if self.return_labels:
            label = self.labels[idx]
            return image, torch.tensor(label)
        else:
            return image

In [8]:
# new_clusters

In [9]:
data_cleaning = {
    'f75f7082d62a44e7bfd373532877c9a6': 36,
    '552656baa25848009b9eff3dfbedd15c': 10,
    '96ebf9d10efa4804a5580f4d55e64d38': 16,
    '3b9ba5ab3fce43c2bca84ee29e77c203': 39,
    '726df8d4f65f4855849925746d01f0ae': 27,
    '325cdb0bf6694d879a0f3518178feb60': 29,
    'a1a79389069f4ddd86bfbdec7078ed02': 19,
    '8d1c755d31a6484686c4a98ad45ab0e7': 29,
    '499c8686053d47ca8395c4f5b7780bd0': 37,
    '160dfcfe3c4e4623a3e9c360da87891c': 19,
    '6b9c4d2192c74d82bbe02d52c5b1e007': 16,
    '9fdbdc45cb7b4e53b0efaab769f7e224': 18,
    '403446096ccf434f9e096b8291c40ac9': 18,
    '23db675a000249498062844cdb31b76f': 18,
    'd6367566a1dc445b87600b8c98ea5402': 18,
    '980f8122f0db49cc94eef626826f3614': 22,
    'd9d0792cefa74c55bdef834be6d34653': 20,
    '670eb7df1cf54be09e18bc13496a50f1': 36,
    '427ee587bd854410ada20b619b186f2b': 26,
    '7bde72d406024ec0b7e72ca9e1edb311': 27,
    '334b17efee804bfd82bb4189c3331ddb': 20,
    'd94c01292d6f4fc19873234ccd85f091': 21,
    '68b0cc47f2034ec98a3db6bf8b9df8e5': 13,
    'afd68eec35824484b63a730ee8a19bef': 16,
    '9e3d20f8c75f4dbea6085c514c4afb48': 7
}

In [59]:
# Clean data
image_paths = Path('test-task/clusters/')
data = pd.read_csv('test-task/clusters.csv', index_col='Unnamed: 0')

new_clusters = defaultdict(list)

for cluster, thresh_idx in data_cleaning.items():
    images = list(data[data.cluster_id == cluster].file_name)
    new_clusters[cluster] = images[:thresh_idx]
    new_clusters['trash'].extend(images[thresh_idx:])
    
new_data = []
for cluster_id, images in new_clusters.items():
    new_data.extend([[cluster_id, image] for image in images])

data = pd.DataFrame(new_data, columns = ['cluster_id', 'file_name'])
num_classes = len(data.cluster_id.unique())

encoder = LabelEncoder()
data['label'] = encoder.fit_transform(data['cluster_id'])

X_train_df, X_val_df, y_train_df, y_val_df = train_test_split(data[['cluster_id', 'file_name']], 
                                                              data['label'], stratify=data['label'], 
                                                              test_size=0.2, random_state=42)
train_df = pd.concat([X_train_df, y_train_df], axis=1)
val_df = pd.concat([X_val_df, y_val_df], axis=1)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
# test_df.reset_index(drop=True, inplace=True)

print('num_classes', num_classes)
print(train_df.shape)
print(val_df.shape)
# print(test_df.shape)

train_dataset = ImageDataset(image_dir=image_paths, csv_layout=train_df, transforms=get_train_transforms())
val_dataset = ImageDataset(image_dir=image_paths, csv_layout=val_df, transforms=get_valid_transforms())
# test_dataset = ImageDataset(image_dir=image_paths, csv_layout=test_df, transforms=get_valid_transforms())

num_classes 26
(1216, 3)
(305, 3)


In [15]:
train_df.label.unique().difference(set(val_dataset.labels))

AttributeError: 'numpy.ndarray' object has no attribute 'difference'

# Инициилизирует необходимое и обучим модель

In [12]:
# Возьмем Effientnet в качестве базовой модели и заметим слой классификации на тождественное преобразование
# То есть на выходе будем получать сырые фичи из бэкбона
trunk = efficientnet_b3(weights=EfficientNet_B3_Weights.DEFAULT)
trunk_output_size = trunk.classifier[1].in_features
trunk.classifier = nn.Identity()
trunk = torch.nn.DataParallel(trunk.to(device))

# Сделаем простенький эмбеддер, который будет преобразовывать фичи из бэкбона в наше латентное пространство
# Приемлемое качество далось на размерности пространства (финального эмбеддинга) = 512
simple_embedder = nn.Sequential(nn.Linear(trunk_output_size, embedding_size))
embedder = torch.nn.DataParallel(simple_embedder.to(device))

# Set optimizers
trunk_optimizer = torch.optim.Adam(trunk.parameters(), lr=0.00001, weight_decay=0.0001)
embedder_optimizer = torch.optim.Adam(
    embedder.parameters(), lr=0.0001, weight_decay=0.0001
)

В данной реализации задачи я взяла библиотеку `pytorch_metric_learning`, в ней реализованы интересные фичи для визуализации нашего пространства.
В остальном библиотека содержит обертки над стандартными классами `pytorch`, а также популярные лоссы для метрического обучения.


Я взяла `TripletMarginLoss`, так как это один из базовых лоссов для разделения классов + он имеет margin, который важен в нашем случае очень маленького размера класса. Более сложные лоссы вроде cosface/arcface использовать в данной задаче смысла не вижу, к тому же домен фотографий в нашем случае сильно отличается от домена лиц, в котором решаются специфические задачи с помощью arcfase лосса

In [13]:
# Лосс функция
loss = losses.ArcFaceLoss(num_classes=num_classes, embedding_size=embedding_size, margin=0.3, scale=5)

# Функция создания пар во время обучения
miner = miners.MultiSimilarityMiner(epsilon=0.1)

# Сэмплер данных
sampler = samplers.MPerClassSampler(
    train_dataset.labels, m=5, length_before_new_iter=len(train_dataset)
)

# Формируем финальные параметры для тренировки
models = {
    "trunk": trunk, 
    "embedder": embedder
}
optimizers = {
    "trunk_optimizer": trunk_optimizer,
    "embedder_optimizer": embedder_optimizer,
}

loss_funcs = {"metric_loss": loss}
# mining_funcs = {"tuple_miner": miner}

### Вспомогательные методы для визуализации пространства, а также тестирование и сохранение лучших весов модели

In [15]:
from pytorch_metric_learning.utils import logging_presets

record_keeper, _, _ = logging_presets.get_record_keeper("logs", "tensorboard")
hooks = logging_presets.get_hook_container(record_keeper)
dataset_dict = {"val": val_dataset}
model_folder = "saved_models"


def visualizer_hook(umapper, umap_embeddings, labels, split_name, keyname, *args):
    logging.info(
        "UMAP plot for the {} split and label set {}".format(split_name, keyname)
    )
    label_set = np.unique(labels)
    num_classes = len(label_set)
    plt.figure(figsize=(10, 7))
    plt.gca().set_prop_cycle(
        cycler(
            "color", [plt.cm.nipy_spectral(i) for i in np.linspace(0, 0.9, num_classes)]
        )
    )
    for i in range(num_classes):
        idx = labels == label_set[i]
        plt.plot(umap_embeddings[idx, 0], umap_embeddings[idx, 1], ".", markersize=1)
    plt.show()


# Тестировщик на этапе валидации модельки, после подсчета метрик строится векторное пространство
tester = testers.GlobalEmbeddingSpaceTester(
    end_of_testing_hook=hooks.end_of_testing_hook,
    visualizer=umap.UMAP(),
    visualizer_hook=visualizer_hook,
    dataloader_num_workers=dataloader_num_workers,
    accuracy_calculator=AccuracyCalculator(k="max_bin_count"),
)

end_of_epoch_hook = hooks.end_of_epoch_hook(
    tester, dataset_dict, model_folder, test_interval=1, patience=1
)

In [19]:
trainer = trainers.MetricLossOnly(
    models=models,
    optimizers=optimizers,
    batch_size=batch_size,
    loss_funcs=loss_funcs,
    dataset=train_dataset,
    mining_funcs={},
    sampler=sampler,
    dataloader_num_workers=0,
    end_of_iteration_hook=hooks.end_of_iteration_hook,
    end_of_epoch_hook=end_of_epoch_hook,
)

In [20]:
trainer.train(num_epochs=num_epochs)

INFO:PML:Initializing dataloader
INFO:PML:Initializing dataloader iterator
INFO:PML:Done creating dataloader iterator
INFO:PML:TRAINING EPOCH 1
total_loss=3.26221:   1%|▉                                                                                     | 1/97 [00:20<32:50, 20.52s/it]


KeyboardInterrupt: 

### Заметки по обучению

1. Обучение сходится за 3 эпохи с хорошими целевыми метриками (показаны ниже) на отложенной выборке, это значит, что модель достаточно хорошо обобщается на наших данных без переобучения
2. Анализируя векторное пространство можно заметить, что вектора почти не формируются в плотные кластера, что верно описывает наши разреженные данные. Пространство достаточно равномерное. Конечно, из-за маленького размера кластера не представляется возможным оценить false positive эмбеддинги, то есть неверно сближенные.

## Искусственно создадим тестовую выборку и посчитаем precision, accuracy, recall, f1-score

Для этого возьмем отложенную тестовую выборку и создадим из нее две части. В сумме размер теста сделаем 1000 пар:
1. Пары, где картинки из одного класса, попавшие при train_test разбиении в тестовую часть. Их фиксированное число, пусть будет `n`
2. Пары, где картинки из разного класса. Рандомно выберем номера instance для каждой пары. Таких пар будет `1000 - n`

Очевидно, что при таком подходе создании пар непохожих изображений качество может немного меняться, однако если модель устойчива, то качество меняется не стат. значимо. Для бОльшей уверенности в результатах можно генерировать тестовую выборку несколько раз и усреднять результаты. Я проверяла такой поход на маленьком тест-сете, чтобы убедиться в верности своих рассуждений, однако в итоговых результатах я привожу значения целевых метриках на одном полноценном прогоне в связи с временными ограничениями.

In [9]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

from torch.nn import functional
import random

In [57]:
def get_samples_from_class(dataset: ImageDataset, idx: int) -> None:
    images = dataset.label_to_images[idx]
    image_path = dataset.image_dir
    pairs = np.array_split(images, int(len(images) / 2))

    for pair in pairs:
        path1 = os.path.join(image_path, pair[0])
        path2 = os.path.join(image_path, pair[1])
        
        image1 = open(path1,'rb').read()
        image2 = open(path2, 'rb').read()

        wi1 = widgets.Image(value=image1, format='jpg', width=300, height=400)
        wi2 = widgets.Image(value=image2, format='jpg', width=300, height=400)
        a = [wi1, wi2]
        wid = widgets.HBox(a)
        print(pair[0], pair[1])
        display.display(wid)


In [76]:
# val_dataset.images_paths

In [78]:
open(val_dataset.images_paths[0])

<_io.TextIOWrapper name='cyprus_testcase/CYPRUS_DATA/clusters/a834306344194287b0311ed05accdbcd.jpg' mode='r' encoding='UTF-8'>

In [85]:
def show_two_images_by_idx(dataset, idx1, idx2):
    image_path = dataset.image_dir
    
    path1 = dataset.images_paths[idx1]
    path2 = dataset.images_paths[idx2]

    image1 = open(path1,'rb').read()
    image2 = open(path2, 'rb').read()

    wi1 = widgets.Image(value=image1, format='jpg', width=300, height=400)
    wi2 = widgets.Image(value=image2, format='jpg', width=300, height=400)
    a = [wi1, wi2]
    wid = widgets.HBox(a)
    print(path1, path2)
    display.display(wid)

In [86]:
show_two_images_by_idx(val_dataset, 0, 5)

cyprus_testcase/CYPRUS_DATA/clusters/a834306344194287b0311ed05accdbcd.jpg cyprus_testcase/CYPRUS_DATA/clusters/1eec91c1ab8b4498a83afd88e65519dd.jpg


In [74]:
get_samples_from_class(val_dataset, 3)

b46d0850ae334f198cdb6d39c61ae939.jpg b18d3749990c4a26a04c9d280deea40d.jpg


3f18414c699e4967aa5814560f99fe1b.jpg 258c948b871049579015be0846eec208.jpg


13a11a4b24614be7b7cfde398ffcd9e5.jpg 0dfa00218e0e44cb904efcb1bf76faae.jpg


13adbaf02dc44e31a7272df6256a7e7f.jpg 0de1d340f8674816a03e05601c9978f4.jpg


f253dab885824dbca2cdb3cc340bd5e1.jpg e01d8019aa9a4e47a6b29051668cb272.jpg


...

d7596efc1adb4625aa454c2ab090139f.jpg f4f90dd94405430dae3c35e6cda27c66.jpg


ead79824119a498a98b1952687851dc4.jpg c1e5f24e7b46479c9550ee9b6acee582.jpg


7f0300ba8ef343b2ac420299c11df922.jpg d6bb7edc5fe8446bae64217de5da6a14.jpg


0d7e14ed9f4e4466abf1c6b64280ca45.jpg 1dd1ccc7d1d44d3f9d1b221f693714a8.jpg


2af9acaa67cb418e94c0c750199844dc.jpg 3c2ad926934845f789649e2311ce3dbb.jpg


In [67]:
# get_samples_from_class(val_dataset, 10)

In [11]:
# Создадим экземляр инференса, по сути это просто обертка над двумя сетками, которая возвращает net2(net1(tensor))
inference_model = InferenceModel(
    trunk=trunk,
    embedder=embedder,
    normalize_embeddings=True,
)

In [13]:
import random

In [14]:
# Cоберем все существующие пары, которые попали в тест
n_samples = 500

pairs = []
targets = [0] * n_samples
dupli = val_df.label[val_df.label.duplicated() == True]
dupli_labels = list(set(dupli))

for l in dupli_labels:
    idxs = list(val_df.label[val_df.label == l].index)
    idxs_full = [(idxs[i], idxs[j]) for i in range(len(idxs)) for j in range(i + 1, len(idxs))]

    pairs.extend(random.choices(idxs_full, k=len(idxs)))
    
targets[:len(pairs)] = [1] * len(pairs)
    
print('Кол-во пар идентичных изображений:', len(pairs))
print('Кол-во пар различных изображений:', n_samples - len(pairs))


# соберем разные изображения, т.е. instance/label разные
n_labels = len(set(val_df.label))

for _ in range(n_samples - len(pairs)):
    idx1, idx2 = random.choice(val_df.label), random.choice(val_df.label)
    while idx1 == idx2 or val_df.label[idx1] == val_df.label[idx2]:
        idx1, idx2 = random.choice(val_df.label), random.choice(val_df.label)
        
    pairs.append([idx1, idx2])
    
print('Кол-во пар идентичных изображений + различных изображений:', len(pairs))
assert len(pairs) == len(targets)

Кол-во пар идентичных изображений: 305
Кол-во пар различных изображений: 195
Кол-во пар идентичных изображений + различных изображений: 500


In [222]:
# distances = []
# for idx1, idx2 in tqdm(pairs):
#     _, dist = is_match(val_dataset, idx1, idx2, return_dist=True)
#     distances.append(dist)
    
# for dist, (idx1, idx2) in zip(distances, pairs):
#     if dist > 0.7:
#         print(dist, val_dataset.labels[idx1], val_dataset.labels[idx2])
#         show_two_images_by_idx(val_dataset, idx1, idx2)

In [23]:
pairs[:10]

[[6, 7],
 [0, 9],
 [14, 15],
 [14, 15],
 [12, 17],
 [6, 7],
 [0, 9],
 [12, 17],
 [22, 24],
 [11, 25]]

In [40]:
# targets

In [17]:
# Незамысловая функция сравнения двух эмбеддингов
# Для перебора порогов можно получить также расстояние
def is_match(dataset:ImageDataset, idx1, idx2, threshold=0.9, return_dist=False) -> Union[bool, Tuple[bool, float]]:
    embed1 = inference_model.get_embeddings(dataset[idx1][0].unsqueeze(0))
    embed2 = inference_model.get_embeddings(dataset[idx2][0].unsqueeze(0))
    
    # cos ~ [-1, 1] -> (cos + 1) / 2 ~ (0, 1) ~ вероятность идентичности 
    dist = float((torch.nn.functional.cosine_similarity(embed1, embed2) + 1) / 2)
    match = dist > threshold
    
    if return_dist:
        return match, dist
    
    return match

In [18]:
# Получим эмбединги и предсказания по идентичности изображений
# Я успела перебрать 5 трешхолдов и 0.75 дал лучший результат по f1-score для данной модели
preds = np.array([[0] * len(pairs)] * 5)
thresholds = [0.7, 0.75, 0.8, 0.85, 0.9]

for i, (idx1, idx2) in tqdm(enumerate(pairs)):
    _, dist = is_match(val_dataset, idx1, idx2, return_dist=True)
    
    # Переберем в цикле условия по порогам, чтобы не гонять 5 раз сетку для одной и той же пары
    for j in range(len(thresholds)):
        if dist > thresholds[j]:
            preds[j][i] = 1


0it [00:00, ?it/s]

In [258]:
# Целевые метрики
metrics = [accuracy_score, precision_score, recall_score, f1_score]
thresholds = [0.7, 0.75, 0.8, 0.85, 0.9]
for pred, thr in zip(preds, thresholds):
    print(f"Порог: {thr}")
    for m in metrics:
        print(f"    {m.__name__}: {m(targets, pred)}")

Порог: 0.7
    accuracy_score: 0.912
    precision_score: 0.9090909090909091
    recall_score: 0.9508196721311475
    f1_score: 0.9294871794871795
Порог: 0.75
    accuracy_score: 0.926
    precision_score: 1.0
    recall_score: 0.8786885245901639
    f1_score: 0.9354275741710296
Порог: 0.8
    accuracy_score: 0.882
    precision_score: 1.0
    recall_score: 0.8065573770491803
    f1_score: 0.8929219600725953
Порог: 0.85
    accuracy_score: 0.778
    precision_score: 1.0
    recall_score: 0.6360655737704918
    f1_score: 0.7775551102204408
Порог: 0.9
    accuracy_score: 0.606
    precision_score: 1.0
    recall_score: 0.3540983606557377
    f1_score: 0.523002421307506


# Построение knn

In [12]:
from sklearn.metrics import v_measure_score
import torchvision
from PIL import Image

In [34]:
trunk = torchvision.models.efficientnet_b3(weights=torchvision.models.EfficientNet_B3_Weights.DEFAULT)
trunk_output_size = trunk.classifier[1].in_features
trunk.classifier = nn.Identity()
trunk = trunk.to(device)

simple_embedder = nn.Sequential(nn.Linear(trunk_output_size, embedding_size))
embedder = simple_embedder.to(device)


In [22]:
# Создадим экземляр инференса, по сути это просто обертка над двумя сетками, которая возвращает net2(net1(tensor))
inference_model = InferenceModel(
    trunk=trunk,
    embedder=embedder,
    normalize_embeddings=True,
)

In [37]:
# inference_model.train_knn(train_dataset)

In [28]:
inference_model.load_knn_func('knn')

In [38]:
# inference_model.save_knn_func('knn')

In [39]:
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, pin_memory=True)

In [47]:
lst = [
    'test-task/clusters/003e4f79084c425eabb579482388820a.jpg',
    'test-task/clusters/005f02dedd154c0889e84599e73f428f.jpg'
]

In [21]:
images = []
for image_path in lst:
    image = Image.open(image_path)
    image = get_valid_transforms()(image)
    images.append(image)

batch = torch.stack(images)

NameError: name 'lst' is not defined

In [26]:
image.shape[0]

3

In [24]:
image = Image.open('test-task/clusters/003e4f79084c425eabb579482388820a.jpg')
image = get_valid_transforms()(image)

dist, cl = inference_model.get_nearest_neighbors(image, k=1)

ValueError: expected 4D input (got 3D input)

In [ ]:
dists = []
pred_labels = []
for d, idx in zip(distances, indices):
    dists.append(d[0])
    pred_labels.append(

In [ ]:
torch.cat(valid_labels_list, dim=0).numpy()

In [57]:
dist

tensor([[1.6768],
        [1.5984]])

In [52]:
# image = get_valid_transforms()(image)

In [44]:
cl[0][0]

tensor(1038)

In [39]:
# image

In [45]:
inference_model.knn_func.index.

TypeError: 'IndexFlat' object is not callable

In [67]:
valid_labels_list = []
valid_distance_list = []
valid_indices_list = []
preds_labels_list = []

for images, labels in tqdm(val_dataloader):

    distances, indices = inference_model.get_nearest_neighbors(images, k=1)
    valid_labels_list.append(labels[0])
    valid_distance_list.append(distances[0])
    valid_indices_list.append(indices[0][0])
    preds_labels_list.append(train_dataset.labels[indices[0][0]])

valid_labels = torch.cat(valid_labels_list, dim=0).numpy()
valid_distances = torch.cat(valid_distance_list, dim=0).numpy()
valid_indices = torch.cat(valid_indices_list, dim=0).numpy()

  0%|          | 0/305 [00:00<?, ?it/s]

INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality is 256
INFO:PML:running k-nn with k=1
INFO:PML:embedding dimensionality

RuntimeError: zero-dimensional tensor (at position 0) cannot be concatenated

In [65]:
train_dataset.labels[672]

22

In [73]:
# valid_labels_list

In [60]:
valid_labels_list[0][0]

tensor(22)

In [51]:
valid_labels[:5]

array([22, 25,  7, 25,  2])

In [53]:
# val_dataset.image_to_label

In [57]:
train_df.iloc[672].label

22

In [54]:
valid_indices[:5]

array([[ 672],
       [1042],
       [ 407],
       [1176],
       [ 717]])

In [27]:
set(train_dataset.labels)

{0, 2, 4, 7, 8, 12, 14, 15, 19, 24, 25}

In [28]:
set(val_dataset.labels)

{0, 1, 3, 5, 6, 9, 10, 11, 13, 16, 17, 18, 20, 21, 22, 23}

In [24]:
# valid_labels

In [76]:
v_measure_score(['a', 'a'], ['a', 'a'])

1.0

## Целевая функция

В связи с тем, что мы в формате jupyter notebook, я не буду здесь переподгружать все библиотеки и переменные, однако просто отмечу, что в прод-варианте, конечно, инферерс и трейн разнесены, поэтому все важные зависимости, переменные и веса необходимо
- фиксировать в requirments
- параметры и стурктуру модели выносить в файл, например, в YAML
- параметры датасета, пути и варианты сэмплирования выносить в файл
- иметь строгое логирование результатов экспериментов
- визуализацию и историчность, например, в mlflow

In [6]:
import torchvision, functional, torch
device = torch.device('cpu')

In [9]:
trunk = torchvision.models.efficientnet_b3(weights=torchvision.models.EfficientNet_B3_Weights.DEFAULT)
trunk_output_size = trunk.classifier[1].in_features
trunk.classifier = torch.nn.Identity()
trunk = trunk.to(device)

simple_embedder = torch.nn.Sequential(torch.nn.Linear(trunk_output_size, 256))
embedder = simple_embedder.to(device)


In [10]:
trunk_weights = torch.load('saved_models/trunk_10.pth', map_location=torch.device('cpu'))
trunk.load_state_dict(trunk_weights)

embedder_weights = torch.load('saved_models/embedder_10.pth', map_location=torch.device('cpu'))
embedder.load_state_dict(embedder_weights)

<All keys matched successfully>

In [92]:
from tools.image_tools import get_samples_from_class

ModuleNotFoundError: No module named 'tools'

In [90]:
import sys
sys.path.append('/Users/julia/Documents/interview/')

In [91]:
sys.path

['/Users/julia/Documents/interview/',
 '/Users/julia/Documents/interview/tools/',
 './tools',
 'tools',
 '/Users/julia/Documents/interview',
 '/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/usr/local/Cellar/python@3.9/3.9.10/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/Users/julia/venvs/python3.9/lib/python3.9/site-packages',
 '../',
 '.',
 '/Users/julia/Documents/interview/']

In [2]:
import tools

NameError: name 'ImageDataset' is not defined

In [1]:
!pwd

/Users/julia/Documents/interview/InteractiveStandard_testcase
